# Coursera Data Science Capstone - Week 3 Assignment
Developed by: Yongkang Liu  
Created on November 4, 2019.  
Updated on November 4, 2019.

<a name="toc"></a>
# Table of contents

1. [Task 1. Web Scraping](#Q1)

1. [Task 2. Obtain Latitude and Longitude information](#Q2)

1. [Task 3. Explore and Cluster the neighbourhoods in Toronto](#Q3)


<a name="Q1"></a>
## 1. Web Scraping using Beautiful Soup
[Back to ToC](#toc)

Question: Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe


### 1.1. Obtaining HTML document data

*Reference: [Beautiful Soup Document](https://beautiful-soup-4.readthedocs.io/en/latest/)

Beautiful Soup is a Python library for pulling data out of HTML and XML files. The latest version is 4 on the date of November 4, 2019, which works for Python 2.7 and Python 3.2.

In [141]:
# import modules
from bs4 import BeautifulSoup
import requests     # an HTTP client to get the document behind a URL as Beautiful Soup expects a document instead of a URL
#import lxml


In [142]:
# Obtain the HTML document from the URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
data = r.text

In [143]:
# Parser the data
soup = BeautifulSoup(data)
# Find the target table
table = soup.find('table', {'class': 'wikitable sortable'})
#table

In [144]:
# Find all rows in the table
table_rows = table.find_all('tr')   # 'tr' is the table row tag in html
print(f"Got table_rows, type: {type(table_rows)}, size: {len(table_rows)}")

Got table_rows, type: <class 'bs4.element.ResultSet'>, size: 289


In [145]:
# Find the headline
print(f'Check the first row: {table_rows[0]}')

Check the first row: <tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


In [146]:
# extract columns
columns = [th.text.strip('\n') for th in table_rows[0].find_all('th')]  # 'th' is the table header tag
print(f'The headline is {columns}')

The headline is ['Postcode', 'Borough', 'Neighbourhood']


In [147]:
# extract table rows
for rIndex, row in enumerate(table_rows):
    if rIndex > 0:
        row_content = [td.text.strip('\n') for td in row.find_all('td')]   # 'td' is the table cell tag
        print(f'Row {rIndex}: {row_content}')

Row 1: ['M1A', 'Not assigned', 'Not assigned']
Row 2: ['M2A', 'Not assigned', 'Not assigned']
Row 3: ['M3A', 'North York', 'Parkwoods']
Row 4: ['M4A', 'North York', 'Victoria Village']
Row 5: ['M5A', 'Downtown Toronto', 'Harbourfront']
Row 6: ['M5A', 'Downtown Toronto', 'Regent Park']
Row 7: ['M6A', 'North York', 'Lawrence Heights']
Row 8: ['M6A', 'North York', 'Lawrence Manor']
Row 9: ['M7A', "Queen's Park", 'Not assigned']
Row 10: ['M8A', 'Not assigned', 'Not assigned']
Row 11: ['M9A', 'Etobicoke', 'Islington Avenue']
Row 12: ['M1B', 'Scarborough', 'Rouge']
Row 13: ['M1B', 'Scarborough', 'Malvern']
Row 14: ['M2B', 'Not assigned', 'Not assigned']
Row 15: ['M3B', 'North York', 'Don Mills North']
Row 16: ['M4B', 'East York', 'Woodbine Gardens']
Row 17: ['M4B', 'East York', 'Parkview Hill']
Row 18: ['M5B', 'Downtown Toronto', 'Ryerson']
Row 19: ['M5B', 'Downtown Toronto', 'Garden District']
Row 20: ['M6B', 'North York', 'Glencairn']
Row 21: ['M7B', 'Not assigned', 'Not assigned']
Row 22:

### 1.2. Loading data into a DataFrame

Create a Pandas dataframe to store all rows as shown above and use the headline row to name the columns.

In [148]:
lstRow = []
# extract table rows
for rIndex, row in enumerate(table_rows):
    if rIndex > 0:
        row_content = [td.text.strip('\n') for td in row.find_all('td')]
        lstRow.append(row_content)
        #print(f'Row {rIndex}: {row_content}')

In [149]:
import pandas as pd
df_tnt = pd.DataFrame(lstRow, columns=columns)
df_tnt.shape

(288, 3)

In [150]:
df_tnt.Postcode.nunique()  # how many unique Postcodes are seen in the table including the assigned ones

180

In [151]:
df_tnt.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 1.3. Clean and Shape the DataFrame

#### 1.3.1. Remove rows without borough assigned

"Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**."

In [152]:
df_tnt_clean = df_tnt[df_tnt['Borough']!='Not assigned']
df_tnt_clean.shape

(211, 3)

In [153]:
df_tnt_clean.Postcode.nunique()  # how many Postcodes assigned to boroughs

103

In [154]:
df_tnt_clean.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### 1.3.2. Update Neighbourhood names

"If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park."

In [155]:
# Check the neighbors that need to be renamed
df_empty_neighbor = df_tnt_clean[df_tnt_clean['Neighbourhood']=='Not assigned']
df_empty_neighbor

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [156]:
def replace_name(x, y):
    if x == 'Not assigned':
        return y
    else:
        return x

df_tnt_clean['Neighbourhood'] = df_tnt_clean.apply(lambda x : replace_name(x.Neighbourhood, x.Borough), axis=1)
#df_tnt_clean.loc[df_tnt_clean.Neighbourhood=='Not assigned', 'Neighbourhood']=df_tnt_clean['Borough']

C:\WPy-3662\python-3.6.6.amd64\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [157]:
df_tnt_clean.head(10) # double check M7A Queen's Park

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### 1.3.3. Group Neighbourhoods by Postcode

"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table."

In [158]:
# Option 1
#df_tnt_unique = df_tnt_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
# Option 2
df_tnt_unique = df_tnt_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [159]:
df_tnt_unique.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**The final dataframe is here.**

In [160]:
df_tnt_unique.shape

(103, 3)

### This is the end of Assignment Task 1

<a name="Q2"></a>
## 2. Obtain Latitude and Longitude information
[Back to ToC](#toc)

In [161]:
!pip install geocoder

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

postal_code = 'M5G'

numTry = 0
# loop until you get the coordinates
while(lat_lng_coords is None):
    numTry += 1
    #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    g = geocoder.google('Toronto, Ontario')
    lat_lng_coords = g.latlng
    print(f'Try # {numTry}')

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [163]:
g = geocoder.google('Toronto, Ontario')

In [164]:
g

<[REQUEST_DENIED] Google - Geocode [empty]>

#### It seems geocoder query does not work. Following online discussions, I switched to another Python module, pgeocode.

Reference: [pgeocode Documentation (Release 0.1.2)](https://buildmedia.readthedocs.org/media/pdf/pgeocode/latest/pgeocode.pdf)

In [165]:
!pip install pgeocode

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [166]:
import pgeocode
nomi = pgeocode.Nominatim('ca')    # use country code: Canada (ca)

In [167]:
postcode = 'M1B'
obj = nomi.query_postal_code(postcode)  # input postcode
print(f'The postcode area: {postcode} has the latitude {obj.latitude} and the longitude {obj.longitude}')

The postcode area: M1B has the latitude 43.8113 and the longitude -79.193


Use the provided CSV files to obtain the latitude and longitude information

In [169]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')

In [170]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [171]:
df_geo.shape

(103, 3)

Combine the two dataframes to include latitude and longitude information into each postcode area.

In [172]:
df_geo.rename(columns={"Postal Code":"Postcode"}, inplace=True)
df_geo.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [173]:
df_toronto_geo = pd.merge(df_tnt_unique, df_geo, on='Postcode', how='inner')

In [175]:
df_toronto_geo.rename(columns={"Postcode":"Postal Code", "Neighbourhood":"Neighborhood"}, inplace=True)
df_toronto_geo.columns

Index(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [176]:
df_toronto_geo

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### This is the end of Task 2

<a name="Q3"></a>
## 3. Explore and Cluster the neighbourhoods in Toronto
[Back to ToC](#toc)

First, load all necessary modules

In [177]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let us look at the dataset we obtained earlier.

In [180]:
print('The Toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto_geo['Borough'].unique()),
        df_toronto_geo.shape[0]
    )
)

The Toronto dataframe has 11 boroughs and 103 neighborhoods.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [181]:
[latitude, longitude] =[43.6532, -79.3832]  # Toronto
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_geo['Latitude'], df_toronto_geo['Longitude'], df_toronto_geo['Borough'], df_toronto_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto City are 43.6532, -79.3832.


In [183]:
# Check neighborhoods in each Borough
df_toronto_geo.Borough.value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

#### I decided to work with only boroughs that contain the word Toronto

In [185]:
lstBorough = df_toronto_geo.Borough.unique().tolist()
lstSelect = []
print('The Boroughs that include the word "Toronto" in their names are:')
for i in lstBorough:
    if 'Toronto' in i:
        print(i)
        lstSelect.append(i)

The Boroughs that include the word "Toronto" in their names are:
East Toronto
Central Toronto
Downtown Toronto
West Toronto


Obtain a new dataframe containing the neighborhood information of Boroughs having the word of Toronto.

In [186]:
df_toronto_real = df_toronto_geo[df_toronto_geo.Borough.isin(lstSelect)]

In [187]:
df_toronto_real

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [188]:
df_toronto_real.shape

(38, 5)

In [190]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto_real['Latitude'], df_toronto_real['Longitude'], df_toronto_real['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [191]:
# Define Foursquare credentials
CLIENT_ID = 'YJW5KECX5B0BGIFZ0IDM0WKTYXMZZN5ZIAWW4XCYAGP5MSK1' # your Foursquare ID
CLIENT_SECRET = 'AIVEIZGBO1D31YQHFO3IR235V1M1CB300RX3H5NZRM3JVE31' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500
LIMIT = 100

Your credentails:
CLIENT_ID: YJW5KECX5B0BGIFZ0IDM0WKTYXMZZN5ZIAWW4XCYAGP5MSK1
CLIENT_SECRET:AIVEIZGBO1D31YQHFO3IR235V1M1CB300RX3H5NZRM3JVE31


#### Explore Neighborhoods in Toronto

For each neighborhood, we obtain up to 100 venues in it within a radius of 500 meters.

To repeat such operations, first we define a function.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [192]:
toronto_venues = getNearbyVenues(names=df_toronto_real['Neighborhood'],
                                   latitudes=df_toronto_real['Latitude'],
                                   longitudes=df_toronto_real['Longitude']
                                  )

The Beaches 43.67635739999999 -79.2930312
The Danforth West,Riverdale 43.6795571 -79.352188
The Beaches West,India Bazaar 43.6689985 -79.31557159999998
Studio District 43.6595255 -79.340923
Lawrence Park 43.7280205 -79.3887901
Davisville North 43.7127511 -79.3901975
North Toronto West 43.7153834 -79.40567840000001
Davisville 43.7043244 -79.3887901
Moore Park,Summerhill East 43.6895743 -79.38315990000001
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West 43.68641229999999 -79.4000493
Rosedale 43.6795626 -79.37752940000001
Cabbagetown,St. James Town 43.667967 -79.3676753
Church and Wellesley 43.6658599 -79.38315990000001
Harbourfront,Regent Park 43.6542599 -79.3606359
Ryerson,Garden District 43.6571618 -79.37893709999999
St. James Town 43.6514939 -79.3754179
Berczy Park 43.644770799999996 -79.3733064
Central Bay Street 43.6579524 -79.3873826
Adelaide,King,Richmond 43.65057120000001 -79.3845675
Harbourfront East,Toronto Islands,Union Station 43.6408157 -79.38175229999999
Design

Check the size of the resulting dataframe.

In [193]:
print(toronto_venues.shape)
toronto_venues.head()

(1710, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each neighborhood.

In [194]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,86,86,86,86,86,86
"Chinatown,Grange Park,Kensington Market",93,93,93,93,93,93
Christie,17,17,17,17,17,17


Let's find out how many unique categories can be curated from all the returned venues.

In [195]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


#### Analyze each neighborhood

In [196]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [197]:
toronto_onehot.shape

(1710, 234)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [198]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.00,0.000000,0.050000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0

In [199]:
toronto_grouped.shape

(38, 234)

Let's print each neighborhood along with the top 5 most common venues.

In [200]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2          Bar  0.04
3   Restaurant  0.03
4   Steakhouse  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.04
2      Farmers Market  0.04
3          Steakhouse  0.04
4              Bakery  0.04


----Brockton,Exhibition Place,Parkdale Village----
                venue  freq
0      Breakfast Spot  0.09
1         Coffee Shop  0.09
2                Café  0.09
3              Bakery  0.04
4  Italian Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Burrito Place  0.05
3       Garden Center  0.05
4             Brewery  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airp

Let's put that into a pandas dataframe.

First, let's write a function to sort the venues in descending order.

In [201]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [202]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Bakery,Steakhouse,Sushi Restaurant,Asian Restaurant,Burger Joint,American Restaurant
1,Berczy Park,Coffee Shop,Café,Cheese Shop,Farmers Market,Bakery,Steakhouse,Seafood Restaurant,Italian Restaurant,Beer Bar,Cocktail Bar
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Grocery Store,Furniture / Home Store,Burrito Place,Sandwich Place,Restaurant,Stadium,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Bar,Sculpture Garden,Harbor / Marina


#### Cluster the neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [203]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [204]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [205]:
toronto_merged = df_toronto_real

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Cosmetics Shop,Brewery,Bubble Tea Shop,Café,Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Brewery,Steakhouse,Sushi Restaurant,Food & Drink Shop,Fish & Chips Shop,Ice Cream Shop,Movie Theater,Pub
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Convenience Store,Brewery,Seafood Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Gym / Fitness Center,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [207]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

In [208]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Cosmetics Shop,Brewery,Bubble Tea Shop,Café,Restaurant
42,East Toronto,0,Park,Sandwich Place,Brewery,Steakhouse,Sushi Restaurant,Food & Drink Shop,Fish & Chips Shop,Ice Cream Shop,Movie Theater,Pub
43,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Convenience Store,Brewery,Seafood Restaurant,Sandwich Place,Cheese Shop
45,Central Toronto,0,Gym,Convenience Store,Food & Drink Shop,Hotel,Sandwich Place,Breakfast Spot,Clothing Store,Park,Eastern European Restaurant,Electronics Store
46,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Health & Beauty Service,Burger Joint,Café,Chinese Restaurant,Dessert Shop,Diner,Gym / Fitness Center
47,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Gym,Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Diner
48,Central Toronto,0,Playground,Tennis Court,Restaurant,Gym,Airport Food Court,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
49,Central Toronto,0,Pub,Coffee Shop,Pizza Place,Sports Bar,Restaurant,Bagel Shop,Supermarket,Sushi Restaurant,Liquor Store,Fried Chicken Joint
51,Downtown Toronto,0,Coffee Shop,Bakery,Park,Pizza Place,Restaurant,Grocery Store,Pharmacy,Café,Market,Pub
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Bubble Tea Shop,Burger Joint,Men's Store,Café


In [209]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,1,Park,Playground,Trail,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
64,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [210]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2,Gym / Fitness Center,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [211]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Garden,Ice Cream Shop,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [212]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Health Food Store,Trail,Pub,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


#### This is the end of Task 3.

<a name="end"></a>
## End of Notebook
[Back](#toc)